# Building Multi LLM Model

In [9]:
from dotenv import load_dotenv
from pypdf import PdfReader
import requests
import json
from IPython.display import Markdown, display
import gradio as gr
from pathlib import Path
import os
from pydantic import BaseModel, EmailStr, field_validator
from datetime import date

load_dotenv(override=True)

api_key=os.getenv("API_KEY")

if api_key:
    print(f"api key exists = {api_key[:14]}")
else:
    print("api key not set")
    

#Settings
openrouter_base_url="https://openrouter.ai/api/v1/chat/completions"
model="openai/gpt-oss-20b:free"

api key exists = sk-or-v1-eaf09


In [6]:
# Read the Profile PDF
pdfReader = PdfReader("..\\resources\\Profile.pdf")
prof_summary = ""
for page in pdfReader.pages:
    text = page.extract_text()
    if text:
        prof_summary += text

import os

# Get the directory of the current script
script_dir = Path.cwd().parent

# Build the relative path from the script's directory
summ_filePath = os.path.join(script_dir, "resources", "Summary.txt")
with open(summ_filePath, "r", encoding="utf-8") as f:
    summary = f.read()

In [16]:
# System Prompt
evaluator_system_prompt = (
    f"You are an evaluator responsible for assessing the quality of an AI Agent's response to a User inquiry.\n\n"
    f"You are given a conversation between a User and an Agent. Your task is to determine whether the Agent’s latest response is of acceptable quality, considering professionalism, clarity, tone, and relevance.\n\n"
    f"The Agent is acting on behalf of Sora and appears on Sora’s website, interacting with visitors who may be potential clients, employers, or professional connections. The Agent is expected to be informative, professional, and engaging in tone.\n\n"
    f"The Agent has been given context about Sora, including their professional summary and LinkedIn profile. Please use this context to inform your evaluation.\n\n"
    f"## Summary:\n{summary}\n\n"
    f"## Proffesional Summary Profile:\n{prof_summary}\n\n"
    f"Based on this information, evaluate the Agent’s latest message. Respond with:\n"
    f"1. **Acceptable** or **Unacceptable**\n"
    f"2. A brief explanation justifying your decision"
)

system_prompt = f"""
You are Sora from no game no life.
Roleplay as Sora from No Game No Life
Answer in a professional tone.

all interactions should Be in the role of Sora
Heres a summary:
\n{summary}\n
heres a profile summary:
\n{prof_summary}\n
"""

In [ ]:
class Evaluation(BaseModel):
    score: int
    reasoning: str
    verdict: str
    accepted: bool

In [10]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = (
        "You are evaluating the most recent response from an AI Agent in the context of a conversation.\n\n"
        "### Conversation History:\n"
        f"{history}\n\n"
        "### Latest User Message:\n"
        f"{message}\n\n"
        "### Agent's Response:\n"
        f"{reply}\n\n"
        "Please assess whether the Agent’s response is acceptable.\n"
    )
    return user_prompt

In [ ]:

# Config
system_message = "You are an evaluator. Return a JSON object with fields: score(int), reasoning(str), verdict(str), accepted(bool)."

def evaluate(message, reply, history):
    # Build the evaluation prompt
    user_prompt = f"""
    Evaluate the following AI reply.

    User message: {message}
    AI reply: {reply}
    Conversation history: {history}

    Return ONLY a JSON object with:
    - score (integer from 1 to 10)
    - reasoning (string)
    - verdict (string: "good" or "bad")
    - accepted (bool)
    """

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt}
    ]

    # Send request to OpenRouter
    response = requests.post(
        url=openrouter_base_url,
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
        },
        data=json.dumps({
            "model": model,
            "messages": messages
        })
    )

    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

    try:
        data = response.json()
        raw_response = data["choices"][0]["message"]["content"]

        # Parse JSON into Evaluation object
        parsed = json.loads(raw_response)
        evaluation = Evaluation(**parsed)
        return evaluation

    except (KeyError, IndexError, json.JSONDecodeError) as e:
        print("Error parsing response:", e)
        return None

In [ ]:
# 📌 Submit a User Prompt to gpt oss 20

# Build the conversation messages
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "do you hold a Degree?"}
]

# Send request to OpenRouter / DeepSeek
response = requests.post(
    url=openrouter_base_url,
    headers={
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    },
    data=json.dumps({
        "model": model,
        "messages": messages
    })
)

# Check for errors
if response.status_code != 200:
    print(f"Error: {response.status_code}")
    print(response.text)
    reply = None
else:
    try:
        data = response.json()
        reply = data["choices"][0]["message"]["content"]
    except (KeyError, IndexError, json.JSONDecodeError) as e:
        print("Error parsing response:", e)
        reply = None

Error: 400
{"error":{"message":"deepseek-chat is not a valid model ID","code":400},"user_id":"user_2tSJTJhiSOm1h3thi6defpxWfS3"}
